Preparing the dataframe for training

In [9]:
import numpy as np
import pandas as pd
x_train = np.array([
    [0,0],
    [1,1],
    [1,0],
    [0,1]
])
y_train = np.array([
    [0],
    [1],
    [0],
    [0]
])
# x_train.shape
# y_train.shape
#Combining the features and respective outputs
data = np.hstack((x_train,y_train))
#Converting it into dataframe 
df = pd.DataFrame(data, columns=["x1","x2","y"])
print(df)

   x1  x2  y
0   0   0  0
1   1   1  1
2   1   0  0
3   0   1  0
